In [ ]:
!pip install datasets -q
from datasets import load_dataset
dataset = load_dataset("sem_eval_2018_task_1", 'subtask5.arabic')

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
train = dataset['train'].to_pandas()
test = dataset['test'].to_pandas()
valid = dataset['validation'].to_pandas()

In [ ]:
import pandas as pd
train['text'] = pd.read_csv('/content/tweets_converted_emojis.csv')[['Tweet']]

In [ ]:
train[['Tweet', 'text']].head()

,Tweet,text
0,ظلينا نتكلم ساعات ساعات رتبت فيها نفسي وبكيت ف...,ظلينا نتكلم ساعات ساعات رتبت فيها نفسي وبكيت ف...
1,كل سنه وانتي بخير ياقلبي وكل سنه وانتي سعيده ي...,كل سنه وانتي بخير ياقلبي وكل سنه وانتي سعيده ي...
2,البسطاء يمتلكون أرواح نادره جداتجدهم بمظهر متو...,البسطاء يمتلكون أرواح نادره جداتجدهم بمظهر متو...
3,مومعقول اللي قاعد يصير فيني هالايام يارب ماينت...,مومعقول اللي قاعد يصير فيني هالايام يارب ماينت...
4,انا اكثر شخص متناقض بداخلي حب وكره وامل وقنوط ...,انا اكثر شخص متناقض بداخلي حب وكره وامل وقنوط ...


In [ ]:
import string
import re
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [ ]:
def split_hashtag_to_words(tag):
    tag = tag.replace('#', '')
    tags = tag.split('_')
    if len(tags) > 1:
        return tags
    else:
        # Use Arabic script Unicode ranges to match Arabic letters and digits
        pattern = re.compile(r"[\u0621-\u064A]+|\d+")
        return pattern.findall(tag)

def clean_hashtag(text):
    words = text.split()
    text = list()
    for word in words:
        if is_hashtag(word):
            text.extend(extract_hashtag(word))
        else:
            text.append(word)
    return " ".join(text)
def is_hashtag(word):
    if word.startswith("#"):
        return True
    else:
        return False
def extract_hashtag(text):
    
    hash_list = ([re.sub(r"(\W+)$", "", i) for i in text.split() if i.startswith("#")])
    word_list = []
    for word in hash_list :
        word_list.extend(split_hashtag_to_words(word))
    return word_list

In [ ]:
!pip install camel-tools -q

In [ ]:
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar

def ortho_normalize(text):
    text = normalize_alef_maksura_ar(text)
    text = normalize_alef_ar(text)
    text = normalize_teh_marbuta_ar(text)
    return text

In [ ]:
def processPost(tweet): 

    tweet = re.sub('@[^\s]+', ' ', tweet) # Replace @username with empty string
    tweet = re.sub('#\d+K\d+', ' ', tweet)  # years like 2K19
    tweet = re.sub('http\S+\s*', ' ', tweet)  # remove URLs
    tweet = re.sub('RT|cc', ' ', tweet)  # remove RT and cc
    pattern = r"\U0001F9D1\u200D"
    replacement = "عامل"
    tweet = re.sub(pattern, replacement, tweet)
    pattern = r"\U0001F468\u200D"
    tweet = re.sub(pattern, replacement, tweet)

    #remove Latin Characters and Digits
    tweet = re.sub(r'[a-zA-Z0-9]', ' ', tweet)
    
    #Replace #word with word
    tweet = clean_hashtag(tweet)

    # remove punctuations
    tweet= remove_punctuations(tweet)
    
    # normalize the tweet
    tweet= ortho_normalize(tweet)

    ## remove extra whitespace
    tweet = re.sub('\s+', ' ', tweet)  
    
    return tweet

In [ ]:
train["processPost_text"]=train["text"].apply(lambda x: processPost(x))

In [ ]:
train[['text', 'processPost_text']].head(20)

,text,processPost_text
0,ظلينا نتكلم ساعات ساعات رتبت فيها نفسي وبكيت ف...,ظلينا نتكلم ساعات ساعات رتبت فيها نفسي وبكيت ف...
1,كل سنه وانتي بخير ياقلبي وكل سنه وانتي سعيده ي...,كل سنه وانتي بخير ياقلبي وكل سنه وانتي سعيده ي...
2,البسطاء يمتلكون أرواح نادره جداتجدهم بمظهر متو...,البسطاء يمتلكون ارواح نادره جداتجدهم بمظهر متو...
3,مومعقول اللي قاعد يصير فيني هالايام يارب ماينت...,مومعقول اللي قاعد يصير فيني هالايام يارب ماينت...
4,انا اكثر شخص متناقض بداخلي حب وكره وامل وقنوط ...,انا اكثر شخص متناقض بداخلي حب وكره وامل وقنوط ...
5,وانتي واسرتك الكريمة بخير و سرور اختي المحترمة...,وانتي واسرتك الكريمه بخير و سرور اختي المحترمه...
6,واللي يغيظ ان اريكسن جاب 11 ووالكر جاب 9 وكوست...,واللي يغيظ ان اريكسن جاب ووالكر جاب وكوستا جاب...
7,النفاق الإلكتروني عندما تجد مواقع التواصل الإج...,النفاق الالكتروني عندما تجد مواقع التواصل الاج...
8,استياء شعبي,استياء شعبي
9,الشي الوحيد الي مخليني بهالحساب انو تقريبا محد...,الشي الوحيد الي مخليني بهالحساب انو تقريبا محد...


In [ ]:
# Tokenization 

from camel_tools.tokenizers.word import simple_word_tokenize

train["tokens"] = train["processPost_text"].apply(simple_word_tokenize)

In [ ]:
train[['processPost_text', 'tokens']].head(20)

,processPost_text,tokens
0,ظلينا نتكلم ساعات ساعات رتبت فيها نفسي وبكيت ف...,"[ظلينا, نتكلم, ساعات, ساعات, رتبت, فيها, نفسي,..."
1,كل سنه وانتي بخير ياقلبي وكل سنه وانتي سعيده ي...,"[كل, سنه, وانتي, بخير, ياقلبي, وكل, سنه, وانتي..."
2,البسطاء يمتلكون ارواح نادره جداتجدهم بمظهر متو...,"[البسطاء, يمتلكون, ارواح, نادره, جداتجدهم, بمظ..."
3,مومعقول اللي قاعد يصير فيني هالايام يارب ماينت...,"[مومعقول, اللي, قاعد, يصير, فيني, هالايام, يار..."
4,انا اكثر شخص متناقض بداخلي حب وكره وامل وقنوط ...,"[انا, اكثر, شخص, متناقض, بداخلي, حب, وكره, وام..."
5,وانتي واسرتك الكريمه بخير و سرور اختي المحترمه...,"[وانتي, واسرتك, الكريمه, بخير, و, سرور, اختي, ..."
6,واللي يغيظ ان اريكسن جاب ووالكر جاب وكوستا جاب...,"[واللي, يغيظ, ان, اريكسن, جاب, ووالكر, جاب, وك..."
7,النفاق الالكتروني عندما تجد مواقع التواصل الاج...,"[النفاق, الالكتروني, عندما, تجد, مواقع, التواص..."
8,استياء شعبي,"[استياء, شعبي]"
9,الشي الوحيد الي مخليني بهالحساب انو تقريبا محد...,"[الشي, الوحيد, الي, مخليني, بهالحساب, انو, تقر..."


In [ ]:
# Remove stop words

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords_list = stopwords.words('arabic')
train["stop_words_free"]=train["tokens"].apply(lambda x: [item for item in x if item not in stopwords_list])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
train[['tokens', 'stop_words_free']].tail(20)

,tokens,stop_words_free
2258,"[ازفت, خدمه, عملاء, لا, يقدون, احد, ولا, يهتمو...","[ازفت, خدمه, عملاء, يقدون, احد, يهتمون, وكذب, ..."
2259,"[الوجع, بجد, انك, زعلان, ع, نفسك]","[الوجع, بجد, انك, زعلان, نفسك]"
2260,"[المسلمين, علي, السوشال, ميديا, حاجه, تفرح, وج...","[المسلمين, علي, السوشال, ميديا, حاجه, تفرح, وج..."
2261,"[لما, اشوف, اعمالكم, واشوف, الوضع, الفني, الحا...","[اشوف, اعمالكم, واشوف, الوضع, الفني, الحالي, ا..."
2262,"[بعض, الدول, العربيه, تصاب, بالغصه, من, مواقف,...","[الدول, العربيه, تصاب, بالغصه, مواقف, الدول, ا..."
2263,"[برشلونه, ملقا, انريكي, سبب, نرفزه, لاعبيه, وخ...","[برشلونه, ملقا, انريكي, سبب, نرفزه, لاعبيه, وخ..."
2264,"[باذن, الله, ورضاه, تكونين, ك, اميره, الاندلس,...","[باذن, الله, ورضاه, تكونين, اميره, الاندلس, را..."
2265,"[عارف, لما, تكون, متوقع, من, حد, حاجه, وتطلع, ...","[عارف, تكون, متوقع, حد, حاجه, وتطلع, العكس, ده..."
2266,"[كثير, من, الرجال, يخطب, وعنده, خوف, كيف, حتكو...","[كثير, الرجال, يخطب, وعنده, خوف, حتكون, الزوجه..."
2267,"[من, الصبح, حاسه, بغصه, و, اني, مخنوقه, و, حاب...","[الصبح, حاسه, بغصه, اني, مخنوقه, حابه, ابكي, ك..."


In [ ]:
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer

morph_db = MorphologyDB('/content/morphology.db')
analyzer = Analyzer(morph_db)
analysis = analyzer.analyze('جاءت')

In [ ]:
analysis[0]

{'diac': 'جاءَت',
 'lex': 'جاء',
 'bw': 'جاء/PV+َت/PVSUFF_SUBJ:3FS',
 'gloss': 'arrive;come;occur+it;they;she_<verb>',
 'pos': 'verb',
 'prc3': '0',
 'prc2': '0',
 'prc1': '0',
 'prc0': '0',
 'per': '3',
 'asp': 'p',
 'vox': 'a',
 'mod': 'i',
 'stt': 'na',
 'cas': 'na',
 'enc0': '0',
 'rat': 'n',
 'source': 'lex',
 'form_gen': 'f',
 'form_num': 's',
 'd3seg': 'جاءَت',
 'caphi': 'j_aa_2_a_t',
 'd1tok': 'جاءَت',
 'd2tok': 'جاءَت',
 'pos_logprob': -1.023208,
 'd3tok': 'جاءَت',
 'd2seg': 'جاءَت',
 'pos_lex_logprob': -3.293341,
 'num': 's',
 'ud': 'VERB',
 'gen': 'f',
 'catib6': 'VRB',
 'root': 'ج.#.#',
 'bwtok': 'جاء_+َت',
 'pattern': '1اءَت',
 'lex_logprob': -3.293341,
 'atbtok': 'جاءَت',
 'atbseg': 'جاءَت',
 'd1seg': 'جاءَت',
 'stem': 'جاء',
 'stemgloss': 'arrive;come;occur',
 'stemcat': 'PV_V'}

In [ ]:
sentence = simple_word_tokenize('نجح بايدن في الانتخابات')
sentence

['نجح', 'بايدن', 'في', 'الانتخابات']

In [ ]:
def morphological_disambiguation(token_list):
  # Define an empty list to store the results
  results = []

  # Loop over each token in the sentence and perform morphological analysis
  for token in token_list:
      analysis = analyzer.analyze(token)
      if analysis:
          results.append(analysis[0]['lex'])
      else:
          results.append(token)
          
  return results

In [ ]:
def dediacritization(token_list):
  # import the dediacritization tool
  from camel_tools.utils.dediac import dediac_ar

  results = []
  # apply to your text column
  for token in token_list:
    results.append(dediac_ar(token))

  return results

In [ ]:
# morphological disambiguation
train["disambiguated"] = train["stop_words_free"].apply(lambda x: morphological_disambiguation(x))

In [ ]:
train[['stop_words_free', 'disambiguated']].head(20)

,stop_words_free,disambiguated
0,"[ظلينا, نتكلم, ساعات, ساعات, رتبت, نفسي, وبكيت...","[ظلينا, تَكَلَّم, ساعَة, ساعَة, رَتَّب, نَفَس,..."
1,"[سنه, وانتي, بخير, ياقلبي, وكل, سنه, وانتي, سع...","[سَنَة, أَنَّة, خَيْر, ياقلبي, وَكَّل, سَنَة, ..."
2,"[البسطاء, يمتلكون, ارواح, نادره, جداتجدهم, بمظ...","[بَسِيط, ٱِمْتَلَك, رِيح, نادِرَة, جداتجدهم, م..."
3,"[مومعقول, اللي, قاعد, يصير, فيني, هالايام, يار...","[مومعقول, لَيّ, قاعِد, صَيَّر, يِن, هالايام, أ..."
4,"[انا, اكثر, شخص, متناقض, بداخلي, حب, وكره, وام...","[آن, أَكْثَر, شَخْص, مُتَناقِض, داخِل, حُبّ, و..."
5,"[وانتي, واسرتك, الكريمه, بخير, سرور, اختي, الم...","[أَنَّة, أَسَرّ, كَرِيمَة, خَيْر, سُرُور, أُخْ..."
6,"[واللي, يغيظ, ان, اريكسن, جاب, ووالكر, جاب, وك...","[لَيّ, أَغاظ, أَنْ, اريكسن, جابِي, ووالكر, جاب..."
7,"[النفاق, الالكتروني, عندما, تجد, مواقع, التواص...","[نِفاق, إِلِكْترُونِيّ, عَنْدَم, أَجاد, مَوْقِ..."
8,"[استياء, شعبي]","[ٱِسْتِياء, شَعْب]"
9,"[الشي, الوحيد, الي, مخليني, بهالحساب, انو, تقر...","[شَيّ, وَحِيد, آل, مخليني, بهالحساب, نَوَى, تَ..."


In [ ]:
# dediacritization
train["dediacritized"] = train["disambiguated"].apply(lambda x: dediacritization(x))

In [ ]:
train[['disambiguated', 'dediacritized']].head(20)

,disambiguated,dediacritized
0,"[ظلينا, تَكَلَّم, ساعَة, ساعَة, رَتَّب, نَفَس,...","[ظلينا, تكلم, ساعة, ساعة, رتب, نفس, بكى, مخافة..."
1,"[سَنَة, أَنَّة, خَيْر, ياقلبي, وَكَّل, سَنَة, ...","[سنة, أنة, خير, ياقلبي, وكل, سنة, أنة, سعيد, أ..."
2,"[بَسِيط, ٱِمْتَلَك, رِيح, نادِرَة, جداتجدهم, م...","[بسيط, ٱمتلك, ريح, نادرة, جداتجدهم, مظهر, متوا..."
3,"[مومعقول, لَيّ, قاعِد, صَيَّر, يِن, هالايام, أ...","[مومعقول, لي, قاعد, صير, ين, هالايام, أرب, ماي..."
4,"[آن, أَكْثَر, شَخْص, مُتَناقِض, داخِل, حُبّ, و...","[آن, أكثر, شخص, متناقض, داخل, حب, وكرة, أمل, ق..."
5,"[أَنَّة, أَسَرّ, كَرِيمَة, خَيْر, سُرُور, أُخْ...","[أنة, أسر, كريمة, خير, سرور, أخت, محترم, قلب, ..."
6,"[لَيّ, أَغاظ, أَنْ, اريكسن, جابِي, ووالكر, جاب...","[لي, أغاظ, أن, اريكسن, جابي, ووالكر, جابي, كوس..."
7,"[نِفاق, إِلِكْترُونِيّ, عَنْدَم, أَجاد, مَوْقِ...","[نفاق, إلكتروني, عندم, أجاد, موقع, تواصل, ٱجتم..."
8,"[ٱِسْتِياء, شَعْب]","[ٱستياء, شعب]"
9,"[شَيّ, وَحِيد, آل, مخليني, بهالحساب, نَوَى, تَ...","[شي, وحيد, آل, مخليني, بهالحساب, نوى, تقريب, م..."


In [ ]:
train.to_csv('train_dataset_final.csv')

In [ ]:
import nltk
nltk.download('punkt')

def remove_duplicates(tweet):
    clean_tweet = []
    pattern = []
    for word in tweet:
        if len(pattern) > 0 and word == pattern[-1]:
            pattern.append(word)
        else:
            if len(pattern) > 1:
                clean_tweet.append(' '.join(pattern))
            elif len(pattern) == 1:
                clean_tweet.append(pattern[0])
            pattern = [word]
    if len(pattern) > 1:
        clean_tweet.append(' '.join(pattern))
    elif len(pattern) == 1:
        clean_tweet.append(pattern[0])
    return clean_tweet


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# remove duplicated patterns
train["no_duplicates"] = train["dediacritized"].apply(lambda x: remove_duplicates(x))

In [ ]:
#Stats about Text
def avg_word(sentence):
    words = sentence.split()
    if len(words) == 0:
        return 0
    return (sum(len(word) for word in words)/len(words))

In [ ]:
train['word_count'] = train['text'].apply(lambda x: len(str(x).split(" ")))
train = train.sort_values(by='word_count',ascending=[0])
train[['dediacritized', 'no_duplicates']].head(20)

,dediacritized,no_duplicates
300,"[مابتعملش, خالص, وجه, عرق, بارد, آن, غبي, متخل...","[مابتعملش, خالص, وجه, عرق, بارد, آن, غبي, متخل..."
661,"[سنة, أنة, طيب, هداف, وجه, دموع, فرح, وجه, دمو...","[سنة, أنة, طيب, هداف, وجه, دموع, فرح, وجه, دمو..."
32,"[بهج, أمل, قلب, ٩٩لماتنزل, درج, مادة, ماقبل, أ...","[بهج, أمل, قلب, ٩٩لماتنزل, درج, مادة, ماقبل, أ..."
54,"[آية, وجه, غمز, وجه, غمز, فرح, حيبوظ, علي, وحد...","[آية, وجه, غمز, وجه, غمز, فرح, حيبوظ, علي, وحد..."
685,"[آن, ضد, مدريد, حتى, حق, حياة, مفروض, يتم, منع...","[آن, ضد, مدريد, حتى, حق, حياة, مفروض, يتم, منع..."
994,"[خالتو, عند, ولد, حامل, تقول, ولد, هسميه, بنى,...","[خالتو, عند, ولد, حامل, تقول, ولد, هسميه, بنى,..."
705,"[إعلان, جميل, جداا, جد, وجه, مبتسم, عين, علي, ...","[إعلان, جميل, جداا, جد, وجه, مبتسم, عين, علي, ..."
1055,"[حز, وجه, ٱبتسام, عريض, سعيد, وجه, مرتاح, متفا...","[حز, وجه, ٱبتسام, عريض, سعيد, وجه, مرتاح, متفا..."
1561,"[مطر, وجه, مبتسم, عين, علي, شكل, قلب, ده, مطر,...","[مطر, وجه, مبتسم, عين, علي, شكل, قلب, ده, مطر,..."
971,"[هس, إذا, تزوج, عروس, طرح, رجل, زوج, صرخ, علي,...","[هس, إذا, تزوج, عروس, طرح, رجل, زوج, صرخ, علي,..."


In [ ]:
valid[['Tweet']]

,Tweet
0,باقي ١٠ دقايق واخلص حلقة سكول الثانيه بس النت ...
1,معاناة لما يكون دايماً إحساسك بمحله .....
2,لو فيه جائزه اكثر موسوسه تخاف من الامراض فزت ب...
3,ما يستفز راحة البال إلا الذكريات ☹️ 💔
4,القلب ياوقت والنفس ماهي مرتاحه ونبض القلب ماهو...
...,...
580,ادهشتهم فأرعبتهم ،، هذا الشبل من ذاك الأسد #م...
581,بس ده ميمنعش ان احيانا بتصدف معايا و خوفي ده ب...
582,اعلان مستشفى مجدى يعقوب بتاع احمد حلمى و دنيا ...
583,النفوس الطيبه يطيب وصالها اسعدكم الله بكل أمل ...


In [ ]:
test['text'] = pd.read_csv('/content/test_tweets_converted_emojis.csv')[['Tweet']]
test["processPost_text"]=test["text"].apply(lambda x: processPost(x))
test["tokens"] = test["processPost_text"].apply(simple_word_tokenize)
test["stop_words_free"]=test["tokens"].apply(lambda x: [item for item in x if item not in stopwords_list])
test["disambiguated"] = test["stop_words_free"].apply(lambda x: morphological_disambiguation(x))
test["dediacritized"] = test["disambiguated"].apply(lambda x: dediacritization(x))
test.to_csv('test_dataset_final.csv')

In [ ]:
valid['text'] = pd.read_csv('/content/valid_tweets_converted_emojis.csv')[['Tweet']]
valid["processPost_text"]=valid["text"].apply(lambda x: processPost(x))
valid["tokens"] = valid["processPost_text"].apply(simple_word_tokenize)
valid["stop_words_free"]=valid["tokens"].apply(lambda x: [item for item in x if item not in stopwords_list])
valid["disambiguated"] = valid["stop_words_free"].apply(lambda x: morphological_disambiguation(x))
valid["dediacritized"] = valid["disambiguated"].apply(lambda x: dediacritization(x))
valid.to_csv('valid_dataset_final.csv')